In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm

cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
df_jo = pm.df('df_jo')
#df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = pm.sort("df_jo", ['created_at','created_at_fe']).reset_index()
df_jo = df_jo.drop(columns=['index'])

#df_jo = df_jo.drop(columns=['Mes_created_at'])
#df_jo_cp = df_jo.copy()
#df_jo_cp['cr_received_date'] = df_jo_cp.cash_request_received_date
#df_jo.info()
pd.options.display.max_columns = None
#display(df_jo)

df = df_jo 
pm.format_to_dates(df, time_format='s') # 'min','s'
fields = ['id_cr','user_id','created_at','moderated_at','transfer_type','stat_cr' ,'amount','fee',
          'n_fees','n_backs','needs_m_check_recov', 'n_recovery', 'n_incidents', #'created_at_dow',
          'stat_fe','id_fe','created_at_fe','updated_at_fe','reason','money_back_date', 'reimbursement_date',
          'to_reimbur','from_date','to_date', 'charge_moment','type','recovery_status' 
          # 'paid_at', 'to_end',, #,'user_id', 'cr_received_date','recovery_status'
          ]

In [45]:
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by=['user_id', 'created_at'])

# obtenemos la primera fila de cada 'user_id' (la más antigua)
primer_registro = df.groupby('user_id').first()
primer_registro['no_moderat'] = primer_registro['moderated_at'].isnull()
no_moderats = primer_registro.reset_index() #level='user_id')
no_moderats = primer_registro[primer_registro['no_moderat'] == True]
#resultat = no_moderats.reset_index()[['id_cr','id_fe','user_id','created_at','moderated_at','','no_moderat','transfer_type']]
resultat = no_moderats[no_moderats.transfer_type.isin(['regular','instant'])].reset_index()
#display(resultat.sort_values(['created_at']))
#display(resultat[resultat.user_id > 0 ][fields].sort_values(['created_at','created_at_fe']).reset_index(drop=True))
display(resultat[fields].head(4))


,id_cr,user_id,created_at,moderated_at,transfer_type,stat_cr,amount,fee,n_fees,n_backs,needs_m_check_recov,n_recovery,n_incidents,stat_fe,id_fe,created_at_fe,updated_at_fe,reason,money_back_date,reimbursement_date,to_reimbur,from_date,to_date,charge_moment,type,recovery_status
0,21488,78,2020-10-16 07:09:24,NaT,instant,money_back,50.0,5.0,1,1,0,0,0,accepted,13954,2020-10-16 08:55:05,2020-10-16 08:55:05,Instant Payment Cash Request 21488,2020-10-31 18:58:51,2020-10-31 07:10:57,15 days 00:01:32.858786,NaT,NaT,after,instant_payment,nice
1,22842,192,2020-10-21 16:20:48,NaT,instant,money_back,50.0,5.0,1,1,0,0,0,accepted,15608,2020-10-21 16:21:44,2020-10-21 16:21:44,Instant Payment Cash Request 22842,2020-10-30 09:27:50,2020-11-25 11:00:00,34 days 18:39:11.392937,NaT,NaT,after,instant_payment,nice
2,23740,217,2020-10-23 18:13:23,NaT,regular,money_back,25.0,0.0,0,1,0,0,0,cr_regular,0,NaT,NaT,None,2020-11-07 19:42:01,2020-11-07 19:13:21,15 days 00:59:57.563817,NaT,NaT,None,0,nice
3,24420,267,2020-10-25 17:18:50,NaT,instant,money_back,100.0,5.0,1,1,0,0,0,accepted,17482,2020-10-25 17:19:29,2020-10-25 17:19:29,Instant Payment Cash Request 24420,2020-11-25 02:36:07,2020-11-25 23:00:00,31 days 05:41:09.659370,2020-11-22 23:51:00,2020-11-25 23:00:00,after,instant_payment,nice
